## Importing

In [2]:
pip install eli5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107747 sha256=5b4db5b3ebb0fce7a718252cf7f6cb2013718bd8d18aaec154179590bb0c717d
  Stored in directory: /root/.cache/pip/wheels/7b/26/a5/8460416695a992a2966b41caa5338e5e7fcea98c9d032d055c
Successfully built eli5


In [3]:
import eli5

In [4]:
from eli5.sklearn import PermutationImportance

In [5]:
from sklearn.ensemble import RandomForestClassifier

In [6]:
import pickle


In [13]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import PolynomialFeatures
from sklearn import decomposition
from sklearn.model_selection import train_test_split
from sklearn import decomposition
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

import os

from tqdm.auto import tqdm

import shutil

from itertools import groupby

import matplotlib.pyplot as plt

from statistics import mean

import scipy.stats as stats
from scipy.stats import skew
from scipy.stats import kurtosis
from scipy.fftpack import fft

import tensorflow as tf
from tensorflow import keras

import pickle

from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Input, Flatten, Conv1D, MaxPooling1D, Concatenate, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
# path of original data folder
data_path = "/content/drive/MyDrive/ITMO-Master's/Thesis/3rd_semester/Data"
# path of csv files, each file is a 2mins walk for a subject
csvD_path = "/content/drive/MyDrive/ITMO-Master's/Thesis/3rd_semester/Data/csv_files"
# In this file, the Null values are replaved with 0 in the HY scale - Null values were given to healthy control
demographics = "/content/drive/MyDrive/ITMO-Master's/Thesis/3rd_semester/Data/Processed_data/demographics_HYprocessed.csv" 
dem_df = pd.read_csv(demographics)
# parequet folder path
parquet_path = "/content/drive/MyDrive/ITMO-Master's/Thesis/3rd_semester/Data/parquet_files/"

## Data

In [11]:
class Data:
  def __init__(self, prepare_or_get, data_folder, demographics_file, parquet_path):
    '''
    :param prepare_or_get:  1: prepare data , 0: load preloaded datas (npy)
    :param data_folder: path for a folder containing csv files, file for each 2 mins walk (for indivdiual subject)
    :param demographics_file: path for csv demographics file
    :save_path: path where to save/read parquet files after preparing them
    :
    '''

    self.data_path = data_folder
    self.parquet_path= parquet_path
    self.dem_path= demographics_file
    self.scale="HoehnYahr"            # fixed for now
    self.seconds_to_discard = 20      # fixed for now
    self.rows_to_discard = 100 * self.seconds_to_discard
    
    # data in arrays
    self.signals_data = []
    self.cycles_data=[]
    self.statical_data=[]
  

    if (prepare_or_get==1):
      self.prepare_data()
    elif (prepare_or_get==0):
      self.get_data()
  
  def get_data(self):
    self.signals_data_df= pd.read_parquet(self.parquet_path+'signals.gzip')
    self.cycles_data_df=pd.read_parquet(self.parquet_path+'cyclels.gzip')
    self.statical_data_df=pd.read_parquet(self.parquet_path+'statical.gzip')
    self.left_stances_df=pd.read_parquet(self.parquet_path+'left_stances.gzip')
    self.right_stances_df=pd.read_parquet(self.parquet_path+'right_stances.gzip')

  def prepare_data(self):
    # loop over files, each file is a 2 min walk for a single individual
    # files are expected to be in csv format
    self.dem_df = pd.read_csv(self.dem_path)
    for name in tqdm((os.listdir(self.data_path))):
       
      # id of subject
      id = name.split('_')[0]

      # disease level according to 'self.scale'
      level = self.get_pd_level( id)

      # parkinon's patient OR healthy control ?
      if 'Co' in name:  
        parkinson = 0 # Healthy control
      elif 'Pt' in name:
        parkinson = 1 # Parkinson's Patient

      # read and filter one file
      one_walk=self.read_filter(name)

      # sum of values from all the left sensors
      one_walk['Total_Force_Left'] = one_walk[list(one_walk.columns[0:8])].sum(axis=1)
     
      # sum of values from all the right sensors
      one_walk['Total_Force_Right'] = one_walk[list(one_walk.columns[8:16])].sum(axis=1)
      
      # convert to lists
      one_walk_lists=[]*18
      for column in one_walk.columns.values.tolist():
        one_walk_lists.append(one_walk[column].tolist()) 

      ## Left foot related ##
      l_stances, l_strides_time , l_swings_time, l_stances_time, l_indices = self.segment_signal(one_walk_lists[16])
      # Find maximum VGRF at heel strike for each gait cycle 
      l_peaks_heel = self.get_max_per_gait_cycle(one_walk_lists[0],l_indices) # sensor 0 = left heal sensor
      # Find maximum VGRF at toe off for each gait cycle
      l_peaks_toe = self.get_max_per_gait_cycle(one_walk_lists[7],l_indices) # sensor 7 = left toe sensor

      ## right foot related ##
      r_stances, r_strides_time , r_swings_time, r_stances_time, r_indices = self.segment_signal(one_walk_lists[17])
      # Find maximum VGRF at heel strike for each gait cycle 
      r_peaks_heel = self.get_max_per_gait_cycle(one_walk_lists[8],l_indices) # sensor 8 = right heal sensor
      # Find maximum VGRF at toe off for each gait cycle
      r_peaks_toe = self.get_max_per_gait_cycle(one_walk_lists[15],l_indices) # sensor 15 = right toe sensor


      ## Saving data ##

      # Raw Signals
      self.signals_data.append([id,level,parkinson] +one_walk_lists)

      # Statical
      self.statical_data.append( [ id, level, parkinson 
                                , mean (l_swings_time) , np.std(l_swings_time), (mean (l_swings_time) / np.std(l_swings_time) *100)
                                , mean (l_strides_time) , np.std(l_strides_time) , (mean (l_strides_time) / np.std(l_strides_time)*100)
                                , kurtosis(l_strides_time), skew(l_strides_time)
                                , mean (l_stances_time) , np.std(l_stances_time) , ( mean (l_stances_time) / np.std(l_stances_time) *100) 

                                , mean (r_swings_time) , np.std(r_swings_time), (mean (r_swings_time) / np.std(r_swings_time) *100)
                                , mean (r_strides_time) , np.std(r_strides_time) , (mean (r_strides_time) / np.std(r_strides_time)*100)
                                , kurtosis(r_strides_time), skew(r_strides_time)
                                , mean (r_stances_time) , np.std(r_stances_time) , ( mean (r_stances_time) / np.std(r_stances_time) *100)
                                
                                , mean (l_peaks_heel) , np.std(l_peaks_heel) ,mean (l_peaks_toe) , np.std(l_peaks_toe)
                                , mean (r_peaks_heel) , np.std(r_peaks_heel) ,mean (r_peaks_toe) , np.std(r_peaks_toe)
                                  ] )
      
      
      # Segmented Signals
      
      # NEED TO BE MOVED TO SEPERATE FUNCTION
      # only for segmented data, for a trial we need each stance phase with the following 
      # swing phases's time, so here we are adjusting the phases to be equal

      dif_l =  len(l_stances)-len(l_swings_time)
      if (dif_l > 0):
        l_stances=l_stances[:-dif_l]
      elif(dif_l < 0):
        l_swings_time= l_swings_time[:dif_l]

      
      dif_r =  len(r_stances)-len(r_swings_time)
      if (dif_r>0):
        r_stances=r_stances[:-dif_r]
      elif(dif_r < 0):
        r_swings_time = r_swings_time[:dif_r]
       
      self.cycles_data.append( [ id, level, parkinson ] 
                              + [l_stances] + [l_strides_time] + [l_swings_time] + [l_stances_time]
                              + [r_stances] + [r_strides_time] + [r_swings_time] + [r_stances_time]
                              + [l_peaks_heel] + [l_peaks_toe]
                              + [r_peaks_heel] + [r_peaks_toe]
                              )
      

    self.statical_data_to_df()
    self.cycles_data_to_df()
    self.signal_data_to_df()
    ## Get individual stances data frames
    self.ind_stances_to_df()
    ## writing files ##
    self.save_files()

  def get_pd_level(self, subject_id):
    level = self.dem_df[self.scale][dem_df['ID'] == subject_id ].values[0]
    return level

  def read_filter(self,name):
      # Reading each file, skipping 3 columns (time,total right forces & total left forces)
      # For gait cycle segmentation, to eliminate the effect of gait initiation and termenation,
      # the first and last N=20 seconds of VGRF data was discarded.

      one_walk = pd.read_csv(self.data_path + '/' + name,
                             skiprows=list(range(1,self.rows_to_discard+1)) ,
                             skipfooter=self.rows_to_discard,
                             usecols = np.arange(1,17),
                             engine = 'python')  
      
      # Usually, Vertical Ground React Force VGRF values less than 20N are generated from noise
      one_walk.where(one_walk > 20, 0, inplace=True)

      return one_walk

  def segment_signal(self, signal):
      # swing - stance phases repeatly
      phases = [list(g) for k, g in groupby((signal), lambda x:x>0)]
      
      # Deleting the first and last phase, since the first and last 20 seconds were deleted,
      # there's no garanty that the first and last phase are complete
      # Hence, they are being deleted here just for safety

      phases = phases[1:-1]

      # filtering phases that is shorter than 20 becuase must propably it's noise
      phases = [l for l in phases if len(l)>20]

      # indices of the gait cycles
      indices=[0]
      for i in range(1, len(phases),2):
        indices.append(indices[-1]+len(phases[i]) + len(phases[i-1]))

      # stances phases
      stances = [l for l in phases if any(l)]
      
      # strides times 
      strides_time =  [((len(phases[i]) + len(phases[i-1]) -1)*0.01 ) for i in range(1,len(phases),2)]

      # swings time
      swings_time = [(len(l)-1)*0.01 for l in phases if not all (l)]

      # stance time 
      stances_time = [(len(l)-1)*0.01 for l in stances]

      return stances, strides_time , swings_time, stances_time, indices

  def get_max_per_gait_cycle(self, signal,indices):
    gait_cycles = self.get_gait_cycles(signal, indices)
    peaks = [max(cycle) for cycle in gait_cycles]
    return peaks

  def get_gait_cycles(self, signal, indices):
    # get gait cycles
    gait_cycles= [signal[indices[i]:indices[i+1]] for i in range(len(indices)-1)]
    return gait_cycles

  def signal_data_to_df(self):
    self.signals_data_df = pd.DataFrame(self.signals_data, columns =["ID", "level", "y", "L1", "L2","L3", "L4","L5","L6","L7","L8",
                                       "R1", "R2","R3", "R4","R5","R6","R7","R8","Total_Force_Right","Total_Force_Left" ])
     
    
  def cycles_data_to_df(self):
    self.cycles_data_df = pd.DataFrame(self.cycles_data, columns =["ID", "level", "y", 
                                                                   "l_stances", "l_strides_time", "l_swings_time" , "l_stances_time"
                                                                  , "r_stances" , "r_strides_time" , "r_swings_time" , "r_stances_time"
                                                                  , "l_peaks_heel" , "l_peaks_toe"
                                                                  , "r_peaks_heel" , "r_peaks_toe" ])
    

  def statical_data_to_df(self):
    self.statical_data_df = pd.DataFrame(self.statical_data, columns =["ID", "level", "y"
                                                                      , "mean_left_swings_time" , "std_left_swings_time", "cv_left_swings_time"
                                                                      , "mean_left_stride_time" , "std_left_stride_time" ,"cv_left_stride_time"
                                                                      , "kurtosis_left_strides_time", "skew_left_strides_time"
                                                                      , "mean_left_stance_time" , "std_left_stance_time", "cv_left_stance_time"
                                                                      , "mean_right_swings_time" , "std_right_swings_time", "cv_right_swings_time"
                                                                      , "mean_right_stride_time" , "std_right_stride_time", "cv_right_stride_time"
                                                                      , "kurtosis_right_strides_time", "skew_right_strides_time"
                                                                      , "mean_right_stance_time" , "std_right_stance_time", "cv_right_stance_time"
                                                                      , "mean_left_peaks_heel" , "std_left_peaks_heel"
                                                                      , "mean_left_peaks_toe" , "std_left_peaks_toe"
                                                                      , "mean_right_peaks_heel" , "std_right_peaks_heel"
                                                                      , "mean_right_peaks_toe" , "std_right_peaks_toe"
                                                                       ])
  def ind_stances_to_df(self):
    #droping NNOT WORKING HERE
    self.left_stances_df = self.cycles_data_df.explode(["l_stances","l_swings_time"])
    self.left_stances_df = self.left_stances_df.drop(columns=["r_stances" ,	"r_strides_time" ,	"r_swings_time" ,	"r_stances_time", "r_peaks_heel" ,	"r_peaks_toe"], axis=1)

    self.right_stances_df = self.cycles_data_df.explode(["r_stances","r_swings_time"])
    self.right_stances_df = self.right_stances_df.drop(columns=["l_stances" ,	"l_strides_time" ,	"l_swings_time" ,	"l_stances_time", "l_peaks_heel" ,	"l_peaks_toe"], axis=1)

  def save_files(self):
    self.signals_data_df.to_parquet(self.parquet_path+'signals.gzip')
    self.cycles_data_df.to_parquet(self.parquet_path+'cyclels.gzip')
    self.statical_data_df.to_parquet(self.parquet_path+'statical.gzip')
    self.left_stances_df.to_parquet(self.parquet_path+'left_stances.gzip')
    self.right_stances_df.to_parquet(self.parquet_path+'right_stances.gzip')

In [15]:
data = Data( 0, "", "",parquet_path)

In [16]:
type(data.signals_data_df["L1"].iloc[-1]) 

numpy.ndarray

FWHM

In [17]:
def fwhm(y_values_temp, x_values):
    y_values, temp_l, temp_r = [], [], []
    ## print(y_values_temp, x_values)
    # To make 'y_values_temp', a numpy array, into a python list
    for x in range(0,len(y_values_temp)):
        y_values.append(y_values_temp[x])
    peak_height = max(y_values)
    half_peak_height = max(y_values)/2
    
    # Splitting the y_values data into before and after x_value at peak height
    y_l_temp = y_values[0:y_values.index(peak_height)]
    y_r_temp = y_values[y_values.index(peak_height):len(y_values)]
    
    # Find 1st closest value to half_peak_height in y_l and y_r
    y_l = [abs(x-half_peak_height) for x in y_l_temp] # the distances
    y_l_min = min(y_l)
    point_left=y_l_temp[y_l.index(y_l_min)]  
    
    y_r = [abs(x-half_peak_height) for x in y_r_temp] # the distances
    point_right=y_r_temp[y_r.index(min(y_r))]  
    
    # y_l = nsmallest(1, y_l_temp, key=lambda x: abs(x-half_peak_height))
    # y_r = nsmallest(1, y_r_temp, key=lambda x: abs(x-half_peak_height))
    
    # Gets x_value pairs for y_l and y_r
    temp_l.append(x_values[y_l_temp.index(point_left)])
    temp_r.append(x_values[y_r_temp.index(point_right)])
    fwhm_n = temp_l[0] - temp_r[0]
    return abs(fwhm_n)

In [18]:
### SHOULD be applied for all dataframes after `explode`
data.right_stances_df = data.right_stances_df.reset_index(drop=True)
data.right_stances_df['max_peak'] = data.right_stances_df['r_stances'].apply(lambda x: max(x))
data.right_stances_df['FWHM'] = data.right_stances_df['r_stances'].apply(lambda x: fwhm(x, np.arange(0, len(x)*0.01,0.01)))

padding

In [25]:
max_length = max([len(row) for row in data.right_stances_df['r_stances']])
padded_data = np.zeros((len(data.right_stances_df['r_stances']), max_length))
for i, row in enumerate(data.right_stances_df['r_stances']):
    padded_data[i, :len(row)] = row
padded_data=np.reshape(padded_data,(len(data.right_stances_df['r_stances']), 1,-1))
padded_data=padded_data.tolist()
padded_data_df = pd.DataFrame(padded_data, columns =["padded_stances"])
data.right_stances_df['padded_stances']=padded_data_df

train -test

In [26]:
df1 = data.right_stances_df[[ 'padded_stances','r_swings_time','max_peak','FWHM']]
y = data.right_stances_df[[ 'y']]
X_train, X_test, y_train, y_test = train_test_split(df1, y, random_state=42)

Scaling

In [28]:
# create a scaler object
scaler = StandardScaler()

# fit and transform the data
scaled_data = scaler.fit_transform(np.stack(X_train['padded_stances'].values))
scaled_data=scaled_data.tolist()
# X_train['scaled_stances']= pd.DataFrame(scaled_data, columns =["scaled_data"]) # this way caused some values to become Nulll
X_train.insert(0, "scaled_stances", scaled_data, True)

reformatting

In [29]:
data = np.append( np.asarray(X_train['scaled_stances'].tolist()) , np.asarray(X_train[["r_swings_time" ]]),  axis=1)
data = np.append( data , np.asarray(X_train[["max_peak" ]]),  axis=1)
data = np.append( data , np.asarray(X_train[["FWHM" ]]),  axis=1)

test data

In [30]:
scaled_test_data = scaler.transform(np.stack(X_test['padded_stances'].values))
scaled_test_data=scaled_test_data.tolist()
X_test.insert(0, "scaled_stances", scaled_test_data, True)
test_data = np.append( np.asarray(X_test['scaled_stances'].tolist()) , np.asarray(X_test[["r_swings_time" ]]),  axis=1)
test_data = np.append( test_data , np.asarray(X_test[["max_peak" ]]),  axis=1)
test_data = np.append( test_data , np.asarray(X_test[["FWHM" ]]),  axis=1)

## Interpretation

### RF Classifier with 100 estimator on scaled stances with 0.94 accuracy

In [31]:
def get_feature_importance(model, X, y, feature_names):
    perm = PermutationImportance(model, random_state=42).fit(X, y)
    return eli5.show_weights(perm)

In [9]:
# Load the model                                                                                                                                                                                                       
with open("/content/drive/MyDrive/ITMO-Master's/Thesis/3rd_semester/Models/rf_classifier", 'rb') as f:
    rf = pickle.load(f)

In [32]:
# Evaluate the performance of the classifier on the testing set
y_pred = rf.predict(test_data)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.9417011094191864


In [40]:
features = [str(i) for i in range (750)]

In [ ]:
def scorer(model, X, y):
    return model.score(pd.DataFrame(X, columns=features), y)

perm = PermutationImportance(rf, scoring=scorer).fit(test_data, y_test)

In [50]:
 eli5.show_weights(perm)

Weight,Feature
0.0104 ± 0.0012,x747
0.0103 ± 0.0028,x1
0.0031 ± 0.0008,x3
0.0026 ± 0.0009,x2
0.0025 ± 0.0004,x54
0.0022 ± 0.0010,x42
0.0020 ± 0.0009,x11
0.0020 ± 0.0003,x45
0.0019 ± 0.0015,x40
0.0018 ± 0.0015,x41


features from 0 to 746 : the stances

747 Swing time

748 Max Peak

749 FWHM

In [48]:
w = eli5.show_weights(perm)
result = pd.read_html(w.data)[0]
result

,Weight,Feature
0,0.0104 ± 0.0012,x747
1,0.0103 ± 0.0028,x1
2,0.0031 ± 0.0008,x3
3,0.0026 ± 0.0009,x2
4,0.0025 ± 0.0004,x54
5,0.0022 ± 0.0010,x42
6,0.0020 ± 0.0009,x11
7,0.0020 ± 0.0003,x45
8,0.0019 ± 0.0015,x40
9,0.0018 ± 0.0015,x41
